Using the SOXS Python interface, this example shows how to generate photons from two thermal spectra and two $\beta$-model spatial distributions, as an approximation of two galaxy clusters. 

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rc("font", size=18)
import soxs

We want to generate thermal spectra, so we first create a spectral generator using the ``ApecGenerator`` class:

In [ ]:
emin = 0.05 # keV
emax = 20.0 # keV
nbins = 20000 
agen = soxs.ApecGenerator(emin, emax, nbins)

Next, we'll generate the two thermal spectra. We'll set the APEC norm for each to 1, and renormalize them later:

In [ ]:
kT1 = 6.0
abund1 = 0.3
redshift1 = 0.05
norm1 = 1.0
spec1 = agen.get_spectrum(kT1, abund1, redshift1, norm1)

In [ ]:
kT2 = 4.0
abund2 = 0.4
redshift2 = 0.1
norm2 = 1.0
spec2 = agen.get_spectrum(kT2, abund2, redshift2, norm2)

Now, re-normalize the spectra using energy fluxes between 0.5-2.0 keV:

In [ ]:
flux1 = 1.0e-13 # erg/s/cm**2
flux2 = 5.0e-14 # erg/s/cm**2
emin = 0.5 # keV
emax = 2.0 # keV
spec1.rescale_flux(flux1, emin=0.5, emax=2.0, flux_type="energy")
spec2.rescale_flux(flux2, emin=0.5, emax=2.0, flux_type="energy")

We'll also apply foreground galactic absorption to each spectrum:

In [ ]:
n_H = 0.04 # 10^20 atoms/cm^2
spec1.apply_foreground_absorption(n_H)
spec2.apply_foreground_absorption(n_H)

``spec1`` and ``spec2`` are ``Spectrum`` objects. Let's have a look at the spectra:

In [ ]:
fig, ax = spec1.plot(xmin=0.1, xmax=20.0, ymin=1.0e-8, ymax=1.0e-3, 
                     label="$\mathrm{kT\ =\ 6\ keV,\ Z\ =\ 0.3\ Z_\odot,\ z\ =\ 0.05}$")
spec2.plot(label="$\mathrm{kT\ =\ 4\ keV,\ Z\ =\ 0.4\ Z_\odot,\ z\ =\ 0.1}$",
           fig=fig, ax=ax)
ax.legend()

Now what we want to do is associate spatial distributions with these spectra. Each cluster will be represented using a $\beta$-model. For that, we use the ``BetaModel`` class. For fun, we'll give the second ``BetaModel`` an ellipticity and tilt it by 45 degrees (a bit extreme, but it demonstrates the functionality nicely):

In [ ]:
# Parameters for the clusters
r_c1 = 30.0 # in arcsec
r_c2 = 20.0 # in arcsec
beta1 = 2.0/3.0
beta2 = 1.0

# Center of the field of view
ra0 = 30.0 # degrees
dec0 = 45.0 # degrees

# Space the clusters roughly a few arcminutes apart on the sky. 
# They're at different redshifts, so one is behind the other.
dx = 3.0/60.0 # degrees
ra1 = ra0 - 0.5*dx
dec1 = dec0 - 0.5*dx
ra2 = ra0 + 0.5*dx
dec2 = dec0 + 0.5*dx

# Now actually create the models
pos1 = soxs.BetaModel(ra1, dec1, r_c1, beta1, ellipticity=0.5, theta=45.0)
pos2 = soxs.BetaModel(ra2, dec2, r_c2, beta2)

Now, what we want to do is generate energies and positions from these models. We want to create a large sample that we'll draw from when we run the instrument simulator, so we choose a large exposure time and a large collecting area (should be bigger than the maximum of the ARF). To do this, we use the `from_models()` method of the `PhotonList` class to make instances of the latter:

In [ ]:
t_exp = (500.0, "ks")
area = (3.0, "m**2")
cluster1 = soxs.PhotonList.from_models("cluster1", spec1, pos1, t_exp, area)
cluster2 = soxs.PhotonList.from_models("cluster2", spec2, pos2, t_exp, area)

We can quickly show the positions using the `plot()` method of the `PhotonList` instances. For simplicity, we'll only show every 100th event using the ``stride`` argument, and restrict ourselves to a roughly $20'\times~20'$ field of view.

In [ ]:
fig, ax = cluster1.plot([30.0, 45.0], 6.0, marker='.', stride=100, label="Cluster 1")
cluster2.plot([30.0, 45.0], 6.0, marker='.', stride=100, fig=fig, ax=ax, label="Cluster 2")
ax.legend()

Now that we have the positions and the energies of the photons in the `PhotonList`s, we can write them to a SIMPUT catalog, using the `SimputCatalog` class. Each cluster will have its own photon list, but be part of the same SIMPUT catalog:

In [ ]:
# Create the SIMPUT catalog "sim_cat" from the photon lists "cluster1" and "cluster2"
sim_cat = soxs.SimputCatalog("clusters", [cluster1, cluster2])
# Write the catalog to a file "clusters_simput.fits"
sim_cat.write_catalog(overwrite=True)

Finally, we can use the instrument simulator to simulate the two clusters by ingesting the SIMPUT file, setting an output file "evt.fits", setting an exposure time of 50 ks (less than the one we used to generate the source), the "hdxi" instrument, and the pointing direction of (RA, Dec) = (30.,45.) degrees.

In [ ]:
soxs.instrument_simulator("clusters_simput.fits", "evt.fits", (50.0, "ks"), "hdxi", [30., 45.], overwrite=True)

We can use the `write_image()` function in SOXS to bin the events into an image and write them to a file, restricting the energies between 0.5 and 2.0 keV:

In [ ]:
soxs.write_image("evt.fits", "img.fits", emin=0.5, emax=2.0, overwrite=True)

The image below plotted using [APLpy](https://aplpy.github.io/) shows the resulting image:

In [ ]:
import aplpy
fig = aplpy.FITSFigure("img.fits")
fig.show_colorscale(vmin=0.1, vmax=10.0, stretch='log', cmap="viridis")
fig.recenter(30., 45., width=0.1, height=0.1) # Centered on the source center with a width of 6 arcmin